### 1.Import Library

In [1]:
import pandas as pd
from transformers import RobertaForQuestionAnswering, Trainer, TrainingArguments,RobertaTokenizerFast
from datasets import Dataset
import torch

c:\Users\hdang\.virtualenvs\machinelearning-lTTH8rYd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2.Fine-Tuning

> data origin

In [4]:
df_bh=pd.read_csv("../data/data_tinhhuong/csv/bhxh_bhyt_bhtn_data.csv")
df_chinhsach=pd.read_csv("../data/data_tinhhuong/csv/chinhsach_nguoicocong_data.csv")
df_gd_daotao_yte=pd.read_csv("../data/data_tinhhuong/csv/giaoduc_daotao_yte_data.csv")
df_gt_xd_tn_mt=pd.read_csv("../data/data_tinhhuong/csv/giaothong_xaydung_tainguyen_moitruong_data.csv")
df_laodong_tt=pd.read_csv("../data/data_tinhhuong/csv/laodong_tienthuong_data.csv")
df_tc_nh_dt_ct=pd.read_csv("../data/data_tinhhuong/csv/taichinh_nganhang_dautu_congthuong_data.csv")
df_linhvuckhac=pd.read_csv("../data/data_tinhhuong/csv/linhvuckhac_data.csv")

> merge data

In [7]:
# Nối các DataFrame lại thành một DataFrame duy nhất
df_tonghop = pd.concat([df_bh, df_chinhsach, df_gd_daotao_yte, df_gt_xd_tn_mt, df_laodong_tt, df_tc_nh_dt_ct, df_linhvuckhac], ignore_index=True)

df_tonghop = df_tonghop.drop(columns='Index')

# Lưu DataFrame đã nối với việc thêm cột 'index' làm tên cột
df_tonghop.to_csv("../data/data_tinhhuong/csv/tonghop_data.csv", index_label='Index')


> Sumary

In [8]:
df_tonghop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12818 entries, 0 to 12817
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       12816 non-null  object
 1   Time       12818 non-null  object
 2   Question   12818 non-null  object
 3   Situation  12818 non-null  object
 4   Answer     12818 non-null  object
dtypes: object(5)
memory usage: 500.8+ KB


In [152]:
df_tonghop=pd.read_csv("../data/data_tinhhuong/csv/tonghop_data.csv")
df_tonghop.columns

Index(['Index', 'Name', 'Time', 'Question', 'Situation', 'Answer'], dtype='object')

> Chuẩn bị dữ liệu

In [153]:
# Create 'context' and 'answers' columns
df_tonghop['context'] = df_tonghop['Question'] + " " + df_tonghop['Situation']
df_tonghop['answers'] = df_tonghop['Answer']

In [154]:
dataset = Dataset.from_pandas(df_tonghop[['Question', 'context', 'answers']])

In [155]:
dataset[0:5]

{'Question': ['Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT?',
  'Có được truy lĩnh chế độ thai sản?',
  'Khi nào cần xin thêm Giấy chứng nhận nghỉ việc hưởng BHXH?',
  'Đóng BHXH tự nguyện bao lâu thì được lương hưu?',
  'Đóng BHXH bao lâu trước khi sinh thì được hưởng chế độ thai sản?'],
 'context': ['Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT? Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?',
  'Có được truy lĩnh chế độ thai sản? Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/2014, tháng 8/2014 tôi sinh con. Do công ty nợ tiền BHXH nên hết thời gian nghỉ thai sản tôi chưa được thanh toán chế độ. Năm 2021 công ty đã nộp hết số tiền nợ BHXH và giải thể. Xin hỏi, bây giờ tôi làm đơn xin hưởng chế độ thai sản của năm 2014 có được không?',
  'Khi nào cần xin thê

In [179]:
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [192]:
from transformers import RobertaTokenizerFast
from datasets import Dataset

# Định nghĩa hàm tokenize_function
def tokenize_function(examples):
    questions = examples['Question']
    contexts = examples['context']
    answers = examples['answers']
    
    encodings = tokenizer(
        questions,
        contexts,
        truncation='longest_first',
        padding='max_length',
        max_length=256,
        return_offsets_mapping=True
    )
    
    start_positions = []
    end_positions = []
    
    for i, answer in enumerate(answers):
        context = contexts[i]
        start_char = context.find(answer)
        end_char = start_char + len(answer)

        if start_char == -1:
            start_positions.append(0)
            end_positions.append(0)
            continue

        offsets = encodings['offset_mapping'][i]
        start_token = 0
        end_token = 0

        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = idx
            if start < end_char <= end:
                end_token = idx

        if end_token < start_token:
            end_token = start_token
        
        start_positions.append(start_token)
        end_positions.append(end_token)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    print(encodings)
    return encodings

# Khởi tạo tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")

# Dữ liệu mẫu
data = {
    'Question': ["What is the capital of France?"],
    'context': ["The capital of France is Paris, which is known for its landmarks."],
    'answers': ["Paris"]
}

# Tạo dataset từ dữ liệu mẫu
dataset = Dataset.from_dict(data)

# Áp dụng hàm tokenize_function lên dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# In kết quả
print(tokenized_datasets)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.
Map: 100%|██████████| 1/1 [00:00<00:00, 11.56 examples/s]

{'input_ids': [[0, 36274, 2573, 2292, 11674, 34423, 1913, 11058, 7360, 114, 2, 2, 8241, 11674, 34423, 1913, 14594, 2573, 17334, 4, 41832, 3069, 2573, 27481, 17837, 7221, 21353, 12868, 5276, 14453, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0

In [181]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 12818/12818 [00:06<00:00, 1884.51 examples/s]


In [182]:
### 5. Load Model
model = RobertaForQuestionAnswering.from_pretrained("vinai/phobert-base")

### 6. Prepare Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

### 7. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

### 8. Train Model
trainer.train()

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\hdang\.virtualenvs\machinelearning-lTTH8rYd\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/4809 [00:00<?, ?it/s]

KeyboardInterrupt: 